In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import quandl
from datetime import datetime
from datetime import timedelta, date
quandl.ApiConfig.api_key= '1ssLPs8xYzykLMyAj-je'#MY KEY, GET YOUR OWN

In [2]:
df = pd.read_csv("constituents.csv")

In [3]:
df.shape

(505, 3)

In [4]:
df.head()

,Symbol,Name,Sector
0,MMM,3M Company,Industrials
1,ABT,Abbott Laboratories,Health Care
2,ABBV,AbbVie,Health Care
3,ACN,Accenture plc,Information Technology
4,ATVI,Activision Blizzard,Information Technology


In [5]:
set(df.Sector)

{'Consumer Discretionary',
 'Consumer Staples',
 'Energy',
 'Financials',
 'Health Care',
 'Industrials',
 'Information Technology',
 'Materials',
 'Real Estate',
 'Telecommunications Services',
 'Utilities'}

In [6]:
tickers = df["Symbol"][:50] #tickers in this csv

In [7]:
price_df = pd.DataFrame()
for ticker in tickers[0:15]:
    price_df[ticker] = quandl.get("WIKI/" + ticker, start_date=str(datetime(2004, 1, 1).date()), end_date = str(date.today()))["Close"]